In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/jigsaw-data/custom.css
/kaggle/input/jigsaw-data/__notebook__.ipynb
/kaggle/input/jigsaw-data/test_all.csv
/kaggle/input/jigsaw-data/__results__.html
/kaggle/input/jigsaw-data/__output__.json
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/ji

In [2]:
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv", usecols=["comment_text", "toxic"]).fillna("none")
train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv", usecols=["comment_text", "toxic"]).fillna("none")
train2.toxic = train2.toxic.round().astype(int)
train = pd.concat([
    train1[['comment_text', 'toxic']],
    train2[['comment_text', 'toxic']].query('toxic==1'),
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)
])
valid = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv")
test = pd.read_csv("/kaggle/input/jigsaw-data/test_all.csv")

In [3]:
import sklearn
train = sklearn.utils.shuffle(train, random_state=42)

In [4]:
maxlen = 192

In [5]:
from transformers import *
MODEL = 'jplu/tf-xlm-roberta-large'
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [6]:
def regular_encode(texts, tokenizer, maxlen=maxlen):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [7]:
%%time
x_train = regular_encode(train.comment_text.astype(str).values, tokenizer, maxlen=maxlen)
x_valid = regular_encode(valid.comment_text.astype(str).values, tokenizer, maxlen=maxlen)
x_test = regular_encode(test.content.astype(str).values, tokenizer, maxlen=maxlen)

x_test_google = regular_encode(test.google.astype(str).values, tokenizer, maxlen=maxlen)
x_test_yandex =regular_encode(test.yandex.astype(str).values, tokenizer, maxlen=maxlen)
x_test_en_blob = regular_encode(test.en_blob.astype(str).values, tokenizer, maxlen=maxlen)

y_valid = valid.toxic.values
y_train = train.toxic.values

CPU times: user 9min 38s, sys: 4.35 s, total: 9min 42s
Wall time: 9min 42s


In [8]:
from numpy import save
save('x_train.npy', x_train)
save('x_valid.npy', x_valid)
save('x_test.npy', x_test)
save('x_test_google.npy', x_test_google)
save('x_test_yandex.npy', x_test_yandex)
save('x_test_en_blob.npy', x_test_en_blob)
save('y_train.npy', y_train)
save('y_valid.npy', y_valid)